In [17]:
!pip install -q -U google-generativeai
!pip install SpeechRecognition
!pip install pyaudio
!pip install pyttsx3
!pip install vadersentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Import packages

In [18]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [19]:
import os
os.getenv('API_KEY')

genai.configure(api_key="API_KEY")

In [20]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [21]:
model = genai.GenerativeModel('gemini-pro')

In [22]:
interview_questions=[
    ["""Why do you want to work in retail?"""
     ,
     """(Most Favorable): I enjoy interacting with people and helping them find what they need. I'm passionate about [company's product/service] and believe in its value to customers. I'm a team player and thrive in a fast-paced environment. (This answer shows enthusiasm, product knowledge, and teamwork skills.)
    (Favorable): I'm looking for a customer-facing role to develop my communication and sales skills. I believe retail offers a dynamic work environment and opportunities for growth. (This answer highlights a desire for skill development and a willingness to learn.)
    (Neutral): I need a job, and retail seems like a good starting point.(This answer is neutral, but doesn't show specific interest in the role.)
    (Less Favorable): My previous job wasn't a good fit, and I'm open to different opportunities. (This focuses on the negative aspects of a past job.)
    (Least Favorable): The hours work well with my schedule.(This answer only focuses on personal needs, not the company or role.)
    """
    ],
    ["""How would you handle a difficult customer?"""
     ,
     """(Most Favorable): "I would stay calm, listen actively to understand their concerns, and apologize for any inconvenience. I would then try to find a solution that meets their needs. If necessary, I would escalate the situation to a manager while remaining courteous to the customer." (This answer emphasizes active listening, problem-solving, and maintaining composure under pressure.)
    (Favorable): "I would try to empathize with the customer's frustration and offer solutions or alternatives. I would also be professional and respectful in my communication." (This answer focuses on empathy, communication skills, and de-escalation.)
    (Neutral): "I would try to reason with the customer and explain the store policy." (This answer suggests a more confrontational approach.)
    (Less Favorable): "I would avoid getting into an argument and just let them vent." (This answer shows a passive approach to conflict resolution.)
    (Least Favorable): "I would ask a coworker to deal with them." (This answer avoids taking responsibility.)
    """
    ],
    ["""You're due to leave at 5 pm but your replacement worker doesn't show up. What would you do?"""
     ,
     """(Most Favorable):"I would first try to contact my replacement to see if they are running late or unable to make their shift. If I can't reach them, I would inform my manager immediately and explain the situation. I would then offer to stay until they can find someone to cover the remaining time, or until a reasonable handover can occur to another coworker. My priority would be to ensure smooth store operations and minimize any inconvenience to customers or colleagues." (This answer shows initiative, responsibility, and teamwork.)
    (Favorable): "I would let my manager know as soon as possible that my replacement isn't there. I would then offer to stay for a short period to help out, but would explain that I have commitments and cannot stay indefinitely." (This answer emphasizes communication and willingness to help, but also sets boundaries.)
    (Neutral): "I would clock out at 5 pm as scheduled." (This answer is neutral, but doesn't show a strong commitment to the team or customer service.)
    (Less Favorable): "I would leave a note for my replacement and clock out at 5 pm." (This answer lacks communication and doesn't ensure a smooth handover.)
    (Least Favorable): "I would just leave without saying anything." (This answer is unprofessional and irresponsible.)
    """
    ],
    ["""A customer wants to return some goods that have been clearly opened and used. How would you handle it?"""
     ,
     """(Most Favorable): "I would politely greet the customer and ask for their receipt. I would then explain the store's return policy regarding opened or used items. If the policy allows for a partial refund or exchange, I would explain those options clearly and professionally. I would also offer to see if there's anything else I can help them with." (This answer demonstrates knowledge of policy, clear communication, and customer service focus.)
    (Favorable): "I would apologize for any inconvenience and ask the customer to see the product. I would then explain the store's return policy and try to find a solution that works for them, such as an exchange for a different item or store credit, if allowed by the policy." (This answer prioritizes customer satisfaction while adhering to policy.)
    (Neutral): "I would ask the customer for their receipt and explain the store's return policy." (This answer is neutral, but lacks details on how you'd handle the specific situation.)
    (Less Favorable): "I would inform the customer that the item cannot be returned because it's opened." (This answer lacks clear explanation and could lead to customer dissatisfaction.)
    (Least Favorable): "That's not possible, you can't return a used item." (This answer is confrontational and offers no solution to the customer.)
    """
    ],
    ["""How do you handle rejection after a customer declines to purchase a product so that you can do better when a new customer arrives?"""
     ,
     """(Most Favorable): "Thank the customer for their time and consideration. Briefly ask if there was anything about the product that didn't meet their needs. This can provide valuable insights for future interactions. Then, politely excuse yourself and move on to the next customer with a positive attitude, focusing on how you can help them." (This answer shows appreciation, seeks feedback for improvement, and maintains a professional demeanor.)

    (Favorable): "Acknowledge the customer's decision and offer assistance with finding alternative products that might better suit their needs. This demonstrates a willingness to help and showcases your product knowledge." (This answer focuses on customer service and suggests alternative solutions.)
    (Neutral): "Thank the customer for their time." (This answer is neutral, but lacks the initiative to learn or improve future interactions.)
    (Less Favorable): "No problem, maybe next time." (This answer shows a lack of enthusiasm and doesn't attempt to learn from the interaction.)
    (Least Favorable): "That's a shame, you're missing out on a great product." (This answer is dismissive and could come across as pressuring the customer.)
    """
    ]
    ]
    

In [23]:
import pyttsx3
def TTS(msg):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)  

    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)  

    text = msg

    engine.say(text)
    engine.runAndWait()


In [27]:
#!pip install speech_recognition
import speech_recognition as sr
#pair=interview_questions[0]
#question= pair[0]

sentiment = []
finalFeedback = []

for question, answer in interview_questions:
    TTS(question)
    # Record audio using microphone
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak Anything...")
        audio = recognizer.listen(source)
    
    # Recognize speech using Google Speech Recognition 
    try:
        text = recognizer.recognize_google(audio)
       # print("You said: {}".format(text))
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

    
    #answer=""" I enjoy interacting with people and helping them find what they need. I'm passionate about the company's work ethic and believe in its value to customers. I'm a team player and thrive in a fast-paced environment."""
    feedback=model.generate_content(question+"Actual answer : "+ answer +"\n Users answer : "+text+
                                    "\n Can you compare the actual answer and the users answer and give feedback in 30 words or less and assign a rating to the answer the user has given on a scale of 1 to 10 .")
    message=feedback.text
    finalFeedback.append(message)
    TTS(message)

    prompt1 = f"""You are an expert at classifying text into either positive, negative or neutral. 
    For every text input that you get, you are to classify it into either positive, neutral or negative.
    The text input is provided between three backticks below.
    ```
    {text}
    ```
    """
    response1 = model.generate_content(prompt1)
    sentiment.append(response1.text)

print(sentiment)
prompt2 = f"""You have just finished interviewig a person from the retail industry, and these are all the feedbacks you have given
            {finalFeedback[0]}\n
            {finalFeedback[1]}\n
            {finalFeedback[2]}\n
            {finalFeedback[3]}\n
            {finalFeedback[4]}\n
            Give a final feedback by summarising all the above feedbacks """
response2 = model.generate_content(prompt2)
message2 = response2.text
TTS(message2)


    

Speak Anything...
Speak Anything...
Speak Anything...
Speech Recognition could not understand audio
Speak Anything...


KeyboardInterrupt: 

In [ ]:

# #answer=""" I enjoy interacting with people and helping them find what they need. I'm passionate about the company's work ethic and believe in its value to customers. I'm a team player and thrive in a fast-paced environment."""
# feedback=model.generate_content(question+"Actual answer : "+pair[1]+"\n Users answer : "+text+
#                                 "\n Can you compare the actual answer and the users answer and give feedback in 30 words or less and assign a rating to the answer the user has given on a scale of 1 to 10 .")
# message=feedback.text
# TTS(message)

In [ ]:
Sentence=(str(text))
analyser=SentimentIntensityAnalyzer()
for i in Sentence:
    v=analyser.polarity_scores(i)
print(v)

In [ ]:
# #!pip install speech_recognition
# import speech_recognition as sr

# # Record audio using microphone
# recognizer = sr.Recognizer()
# with sr.Microphone() as source:
#     print("Speak Anything...")
#     audio = recognizer.listen(source)

# # Recognize speech using Google Speech Recognition
# try:
#     text = recognizer.recognize_google(audio)
#     print("You said: {}".format(text))
# except sr.UnknownValueError:
#     print("Speech Recognition could not understand audio")
# except sr.RequestError as e:
#     print("Could not request results from Google Speech Recognition service; {0}".format(e))

In [ ]:
# !pip install ibm_watson
# import json
# from ibm_watson import SpeechToTextV1
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# # Replace with your IBM Cloud IAM API key
# iam_apikey = "YOUR_API_KEY"

# # Replace with the URL for your Watson Speech to Text service
# url = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/YOUR_SERVICE_INSTANCE_ID"

# # Authenticate
# authenticator = IAMAuthenticator(iam_apikey)
# service = SpeechToTextV1(authenticator=authenticator)

# # Function to recognize speech in real-time from a microphone recording
# def recognize_from_microphone(stream):
#   try:
#     # Set up streaming recognition options
#     recognize_request = SpeechRecognitionResults(service=service)
#     settings = RecognitionSettings(
#       continuous=True,
#       interim_results=True,
#       word_confidence=True
#     )
#     recognize_request.start(settings=settings, audio=stream)

#     # Process the real-time transcription results
#     for transcript in recognize_request.get_result():
#       if transcript.get("results"):
#         for result in transcript["results"]:
#           alternatives = result.get("alternatives", [])
#           if alternatives:
#             text = alternatives[0]["transcript"]
#             confidence = alternatives[0]["confidence"]
#             print(f"You said: {text} (Confidence: {confidence:.2f})")

#     print("Speech recognition completed.")
#   except Exception as e:
#     print("Error recognizing speech:", e)

# # Example usage
# with sr.Microphone() as source:
#   print("Speak now...")
#   recognize_from_microphone(source.stream())
